### Programowanie zorientowane obiektowo i dziedziczenie — związek „jest” - dziedziczenie

In [1]:
class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)
    def work(self):
        print(self.name, "robi różne rzeczy")
    def __repr__(self):
        return f"<Pracownik: {self.name}, wynagrodzenie={self.salary}>"

class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)
    def work(self):
        print(self.name, "przygotowuje jedzenie")
        
class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)
    def work(self):
        print(self.name, "obsługuje klienta")

class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)
    def work(self):
        print(self.name, "przygotowuje pizzę")
        
if __name__ == "__main__":
    bob = PizzaRobot('robert')                # Tworzy robota o imieniu Robert
    print(bob)                                 # Wykonuje odziedziczoną metodę __repr__
    bob.work()                                # Wykonuje działanie specyficzne dla typu
    bob.giveRaise(0.20)                       # Daje robotowi 20-procentową podwyżkę
    print(bob)
    for klass in Employee, Chef, Server, PizzaRobot:
        obj = klass(klass.__name__)
        obj.work()

<Pracownik: robert, wynagrodzenie=50000>
robert przygotowuje pizzę
<Pracownik: robert, wynagrodzenie=60000.0>
Employee robi różne rzeczy
Chef przygotowuje jedzenie
Server obsługuje klienta
PizzaRobot przygotowuje pizzę


### Programowanie zorientowane obiektowo i kompozycja — związki typu „ma”

In [2]:
import pickle 

class Customer:
    def __init__(self, name):
        self.name = name
    def order(self, server):
        print(self.name, "zamawia od", server)
    def pay(self, server):
        print(self.name, "placi za zamowienie", server)
class Oven:
    def bake(self):
        print("piec piecze")
class PizzaShop:
    def __init__(self):
        self.server = Server('Ernest')         # Osadzenie innych obiektów 
        self.chef = PizzaRobot('Robert')       # Robot o imieniu Robert
        self.oven = Oven()
    def order(self, name):
        customer = Customer(name)              # Aktywacja innych obiektów - PizzaShop MA w klientow, pracownikow, piec itp
        customer.order(self.server)            # Klient zamawia od kelnera
        self.chef.work()
        self.oven.bake()
        customer.pay(self.server)
if __name__ == "__main__":
    scene = PizzaShop()                       # Utworzenie kompozytu
    scene.order('Amadeusz')                   # Symulacja zamówienia Amadeusza
    print('...')
    scene.order('Aleksander')                 # Symulacja zamówienia Aleksandra
    pickle.dump(scene, open('scenefile.dat', 'wb'))  # utworzenie pliku pickle i zapisanie danych w formie binarnej

Amadeusz zamawia od <Pracownik: Ernest, wynagrodzenie=40000>
Robert przygotowuje pizzę
piec piecze
Amadeusz placi za zamowienie <Pracownik: Ernest, wynagrodzenie=40000>
...
Aleksander zamawia od <Pracownik: Ernest, wynagrodzenie=40000>
Robert przygotowuje pizzę
piec piecze
Aleksander placi za zamowienie <Pracownik: Ernest, wynagrodzenie=40000>


In [3]:
o = pickle.load(open('scenefile.dat', 'rb'))  # pobranie pliku pickle

In [4]:
o.server

<Pracownik: Ernest, wynagrodzenie=40000>

In [5]:
o.chef

<Pracownik: Robert, wynagrodzenie=50000>

In [6]:
o.order('Zuzanna')

Zuzanna zamawia od <Pracownik: Ernest, wynagrodzenie=40000>
Robert przygotowuje pizzę
piec piecze
Zuzanna placi za zamowienie <Pracownik: Ernest, wynagrodzenie=40000>


In [7]:
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
    def process(self):
        while 1:
            data = self.reader.readline()
            if not data: break
            data = self.converter(data)
            self.writer.write(data)
    def converter(self, data):
        assert 0, 'konwerter musi być zdefiniowany' # lub wywołać wyjątek

In [8]:
class Uppercase(Processor):
    def converter(self, data):
        return data.upper()
    
if __name__ == '__main__':
    import sys
    obj = Uppercase(open('spam.txt'), sys.stdout)
    obj.process()

MIELONKA

In [9]:
class HTMLize:
    def write(self, line):
        print(f'</PRE>{line}</PRE>')

In [10]:
Uppercase(open('spam.txt'), HTMLize()).process()

</PRE>MIELONKA</PRE>


### Programowanie zorientowane obiektowo a delegacja — obiekty „opakowujące”

In [11]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object
    def __getattr__(self, attrname):   #  __getattr__ pobiera nazwę atrybutu w postaci łańcucha znaków
        print('Śledzenie: ', attrname)
        return getattr(self.wrapped, attrname)

In [12]:
X = Wrapper([1,2,3])

In [13]:
X.append(4)

Śledzenie:  append


In [14]:
X.wrapped

[1, 2, 3, 4]

In [15]:
Y = Wrapper({'a':1, 'b':2})

In [16]:
Y.keys()

Śledzenie:  keys


dict_keys(['a', 'b'])

### Pseudoprywatne atrybuty klas:
### zapisuje się je jako *__X* - dwie podlogi przed nazwą atrybutu
### Python automatycznie zniekształca jego nazwę na *_NazwaKlasy__X*
#### np. zmienna *self._X* w klasie *Klasa* otrzyma nazwę *self._Klasa__X*
### ponieważ zmienna zawiera teraz w swojej nazwie również nazwę klasy, jest unikalna, nie będzie wchodziła w konflikt z podobnymi nazwami utworzonymi przez inne klasy hierarchii
###  Nazwy pseudoprywatne zabezpieczają przed przypadkowym przesłonięciem nazwy


### używa się ich np gdy kilku programistow pracuje na tym samym kodzie:
    class C1:
        def m1(self): self.x = 88
        def m2(self): print(self.x)
### a inny programista dopisze swoja klase:
    class C2:
        def m1(self): self.x = 99
        def m2(self): print(self.x)
### i utworzy np klase C3(C1,C2) - w tym wypadku atrybut x wystepuje 2 razy w klasie C3 :/

In [17]:
# ponizej zabezpiecznenie przed taka sytuacja:
class C1:
    def m11(self): self.__x = 88
    def m12(self): print(self.__x)
        
class C2:
    def m21(self): self.__x = 99
    def m22(self): print(self.__x)

In [18]:
class C3(C1, C2): pass

In [19]:
I = C3()

In [20]:
I.m11()

In [21]:
I.m21()

In [22]:
print(I.__dict__)  # dwa niezalezne atrybuty x, ktore nie wchodza ze soba w konflikt

{'_C1__x': 88, '_C2__x': 99}


In [23]:
dir(I)

['_C1__x',
 '_C2__x',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'm11',
 'm12',
 'm21',
 'm22']

In [24]:
I._C1__x

88

In [25]:
I._C1__x = 77 # w ten sposob mozna nadpisac ten nibyprywatny atrybut

### Obiekty metod klas bez wiązania (ang. unbound) — bez *self*
Dostęp do atrybutu funkcji klasy za pomocą składni kwalifikującej z klasą zwraca obiekt
metody bez wiązania. By wywołać metodę, musimy w sposób jawny dostarczyć jej w pierwszym argumencie obiekt instancji. Metoda niezwiązana jest traktowana tak samo jak zwykła funkcja i może być wywołana z poziomu klasy.
### Obiekty metod instancji z wiązaniem (ang. bound) — pary *self + funkcja*
Dostęp do atrybutu funkcji klasy za pomocą składni kwalifikującej z instancją zwraca obiekt
metody z wiązaniem. Python automatycznie pakuje instancję z funkcją w obiekt metody
z wiązaniem, dzięki czemu nie musimy do wywołania tej metody przekazywać instancji.

In [26]:
# przykład, bo nie rozumiem tej definicji z ksiazki:
class Klasa:
    def doit(self, message):
        print(message)

In [27]:
o1 = Klasa()

In [28]:
o1.doit('joł') # zwrócony zostanie obiekt metody z wiązaniem, pakujący instancje o1 z metoda funkcji Klasa.doit

joł


In [29]:
x = o1.doit    # przypisanie obiektu metody z wiązaniem: instancja + funkcja

In [30]:
x('siema')

siema


In [31]:
o2 = Klasa()

In [32]:
t = Klasa.doit      # obiekt metody bez zwiazania - odwolanie do klasy a nie instancji - od tej pory traktowany po prostu jak funkcja

In [33]:
t(o2, 'siemka')     # przekazanie instancji

siemka


In [34]:
class Selfless:
    def __init__(self, data):
        self.data = data
    def selfless(a1, a2):      # jest zwykla funkcja (nie ma self)
        return a1 + a2
    def normal(self, a1, a2):  # jest metoda, oczekuje instancji
        return self.data + a1 + a2

In [35]:
Z = Selfless(2)

In [36]:
Z.normal(2, 4)     # instancja jest przekazana automatycznie

8

In [37]:
Selfless.normal(Z, 2, 4)   # parametr self=Z oczekiwany przez metode przekazany recznie

8

In [38]:
Selfless.selfless(3, 4)   # brak instancji (dziala w pythonie 3.0 ale w 2.6 juz nie)

7

In [39]:
# Z.selfless(3, 4) # nie dziala bo powoduje przekazanie instancji do metody, ktora go nie oczekuje (nie ma agrumentu self)

In [40]:
# Selfless.normal(3,4)  # nie dziala bo nie przekazuje instancji (self) do metody, ktora jej oczekuje

In [41]:
class Number:
    def __init__(self, base):
        self.base = base
    def double(self):
        return self.base * 2
    def triple(self):
        return self.base * 3

In [42]:
x = Number(2)
y = Number(3)
z = Number(4)

In [43]:
x.double()      # zwykle wywolanie bezposrednie

4

In [44]:
x.__dict__

{'base': 2}

In [45]:
xyz = [x.double, y.double, y.triple, z.double]  # lista metod zwiazanych

In [46]:
for a in xyz:    # wywolanie w trybie zwyklych funkcji
    print(a())

4
6
9
8


In [47]:
b = x.double

In [48]:
b.__self__, b.__func__

(<__main__.Number at 0x50e58e0>, <function __main__.Number.double(self)>)

In [49]:
b.__self__.base

2

In [50]:
def kwadrat(arg):
    return arg ** 2

In [51]:
class Sum:
    def __init__(self, val):
        self.val = val
    def __call__(self, arg):
        return self.val + arg

In [52]:
class Iloczyn:
    def __init__(self, val):
        self.val = val
    def metoda(self, arg):
        return self.val * arg

In [53]:
sobject = Sum(2)

In [54]:
pobject = Iloczyn(3)

In [55]:
akcje = [kwadrat, sobject, pobject.metoda]

In [56]:
for a in akcje:
    print(a(5))

25
7
15


In [57]:
[a(5) for a in akcje]

[25, 7, 15]

In [58]:
list(map(lambda a: a(5), akcje))

[25, 7, 15]

In [59]:
class Negate:
    def __init__(self,val):
        self.val = -val
    def __repr__(self):
        return str(self.val)

In [60]:
akcje2 = [kwadrat, sobject, pobject.metoda, Negate]

In [61]:
for a in akcje2:
    print(a(5))

25
7
15
-5


In [62]:
[a(5) for a in akcje2]

[25, 7, 15, -5]

In [63]:
table = {a(5): a for a in akcje2}

In [64]:
for (key, value) in table.items():
    print(f'{key} => {value}')

25 => <function kwadrat at 0x00000000050E7D30>
7 => <__main__.Sum object at 0x0000000005380190>
15 => <bound method Iloczyn.metoda of <__main__.Iloczyn object at 0x00000000053806A0>>
-5 => <class '__main__.Negate'>


### DZIEDZICZENIE WIELOKROTNE - KLASY MIESZANE:
### Wyszukiwanie nazw odbywa się od lewej do prawej, czyli w przypadku klasy C3(C1, C2), python szuka nazwy najpierw w klasie C3, potem gdy jej tam nie znajdzie w C1, potem w C2

### Odczyt listy atrybutów obiektu — *__ dict __()*

In [65]:
class ListInstance:
    """
    Używa __str__() wypisując nazwy i wartości atrybutów jako klasa nadrzędna.
    Używa pseudoprywatnych atrybutów __X w celu uniknięcia konfliktów nazw.
    """
    def __str__(self):
        return f'<Instancja klasy: {self.__class__.__name__}, adres: {id(self)}, \n{self.__attrnames()}>'
    def __attrnames(self):
        result = ''
        for attr in sorted(self.__dict__):
            result += f'\tnazwa: {attr}-{self.__dict__[attr]}\n'
        return result

In [66]:
class Szynka(ListInstance):
    def __init__(self):
        self.data1 = 'mięsko'

In [67]:
S = Szynka()

In [68]:
print(S)

<Instancja klasy: Szynka, adres: 87636192, 
	nazwa: data1-mięsko
>


In [69]:
str(S)

'<Instancja klasy: Szynka, adres: 87636192, \n\tnazwa: data1-mięsko\n>'

In [70]:
S   # nadal domyslna jest metoda repr

In [71]:
class KlasaN:
    def __init__(self):
        self.atrybutklasyN = 'dziedziczony'
    def ham(self):
        pass

In [72]:
class KlasaP(KlasaN, ListInstance):
    def __init__(self):
        KlasaN.__init__(self)
        self.data2 = 'jaja'
        self.data3 = 42
    def spam(self):
        pass

if __name__ == '__main__':
    X = KlasaP()
    print(X)

<Instancja klasy: KlasaP, adres: 84859008, 
	nazwa: atrybutklasyN-dziedziczony
	nazwa: data2-jaja
	nazwa: data3-42
>


In [73]:
class C(ListInstance): pass

In [74]:
x=C()

In [75]:
x.a = 1; x.b = 2; x.c=3

In [76]:
print(x)

<Instancja klasy: C, adres: 84859344, 
	nazwa: a-1
	nazwa: b-2
	nazwa: c-3
>


### Wydobywanie atrybutów odziedziczonych z użyciem dir():
Słownik __ dict__ zawiera jedynie atrybuty instancji, natomiast funkcja dir() dodatkowo zwraca atrybuty odziedziczone.

In [77]:
class ListIn2():
    """
    wykorzystuje dir() do uzyskania atrybutow dziedziczonych i wyprintowania jej dzieki metodzie specjalnej __str__()
    """
    def __str__(self):
        return f'<instancja klasy {self.__class__.__name__}, adres {id(self)}:\n{self.__attrnames()}'
    def __attrnames(self):
        result = ''
        for attr in dir(self):
            if attr[:2] == '__' and attr[-2:] == '__':
                result += f'/tname {attr}=<>\n'   # pomijamy niejawne atrybuty ktore maja __ na poczatku i koncu
            else:
                result += f'/tname {attr}={getattr(self, attr)}\n'
        return result

In [78]:
class SS(KlasaN, ListIn2):
    def __init__(self):
        KlasaN.__init__(self)
        self.data2 = 'jaja'
        self.data3 = 42
    def spam(self):
        pass

if __name__ == '__main__':
    Y = SS()
    print(Y)

<instancja klasy SS, adres 84824176:
/tname _ListIn2__attrnames=<bound method ListIn2.__attrnames of <__main__.SS object at 0x00000000050E5070>>
/tname __class__=<>
/tname __delattr__=<>
/tname __dict__=<>
/tname __dir__=<>
/tname __doc__=<>
/tname __eq__=<>
/tname __format__=<>
/tname __ge__=<>
/tname __getattribute__=<>
/tname __gt__=<>
/tname __hash__=<>
/tname __init__=<>
/tname __init_subclass__=<>
/tname __le__=<>
/tname __lt__=<>
/tname __module__=<>
/tname __ne__=<>
/tname __new__=<>
/tname __reduce__=<>
/tname __reduce_ex__=<>
/tname __repr__=<>
/tname __setattr__=<>
/tname __sizeof__=<>
/tname __str__=<>
/tname __subclasshook__=<>
/tname __weakref__=<>
/tname atrybutklasyN=dziedziczony
/tname data2=jaja
/tname data3=42
/tname ham=<bound method KlasaN.ham of <__main__.SS object at 0x00000000050E5070>>
/tname spam=<bound method SS.spam of <__main__.SS object at 0x00000000050E5070>>



Ponieważ wypisujemy odziedziczone atrybuty, zamiast przeciążać metodę __repr__(), musimy przeciążać __str__(). W przeciwnym razie ten kod spowodowałby zapętlenie: wypisywanie wartości metody wywołuje metodę __repr__() klasy metody.

In [79]:
class Tree:
    def __str__(self):
        self.__visited = {}
        return '<Instancja klasy {0}, adres {1}:\n{2}{3}>'.format(
                                            self.__class__.__name__,
                                            id(self),
                                            self.__attrnames(self, 0),
                                            self.__listclass(self.__class__, 4))
    def __listclass(self, aClass, indent):
        dots = '.' * indent
        if aClass in self.__visited:
            return '\n{0}<Klasa {1}:, adres {2}: (patrz wyżej)>\n'.format(
                           dots,
                           aClass.__name__,
                           id(aClass))
        else:
            self.__visited[aClass] = True
            genabove = (self.__listclass(c, indent+4) for c in aClass.__bases__)
            return '\n{0}<Klas {1}, adres {2}:\n{3}{4}{5}>\n'.format(
                           dots,
                           aClass.__name__,
                           id(aClass),
                           self.__attrnames(aClass, indent),
                           ''.join(genabove),
                           dots)
    def __attrnames(self, obj, indent):
        spaces = ' ' * (indent + 4)
        result = ''
        for attr in sorted(obj.__dict__):
            if attr.startswith('__') and attr.endswith('__'):
                result += spaces + '{0}=<>\n'.format(attr)
            else:
                result += spaces + '{0}={1}\n'.format(attr, getattr(obj, attr))
        return result

In [80]:
class T(KlasaN, Tree):
    def __init__(self):
        KlasaN.__init__(self)
        self.data2 = 'jaja'
        self.data3 = 42
    def spam(self):
        pass

if __name__ == '__main__':
    Z = T()
    print(Z)

<Instancja klasy T, adres 87559760:
    _Tree__visited={}
    atrybutklasyN=dziedziczony
    data2=jaja
    data3=42

....<Klas T, adres 76105792:
        __doc__=<>
        __init__=<>
        __module__=<>
        spam=<function T.spam at 0x0000000005386D30>

........<Klas KlasaN, adres 76100128:
            __dict__=<>
            __doc__=<>
            __init__=<>
            __module__=<>
            __weakref__=<>
            ham=<function KlasaN.ham at 0x00000000050B9430>

............<Klas object, adres 8790742969168:
                __class__=<>
                __delattr__=<>
                __dir__=<>
                __doc__=<>
                __eq__=<>
                __format__=<>
                __ge__=<>
                __getattribute__=<>
                __gt__=<>
                __hash__=<>
                __init__=<>
                __init_subclass__=<>
                __le__=<>
                __lt__=<>
                __ne__=<>
                __new__=<>
            

In [81]:
class ListTree:
    """
    Wykorzystuje __str__() do zwracania drzewa dziedziczenia i atrybutow z kazdego poziomu drzewa
    """
    def __str__(self):
        self.__visited = {}
        return f'<instancja klasy{self.__class__.__name__}, adres {id(self)}: \n{self.__attrnames(self, 0)}{self.__listclass(self.__class__, 4)}'
    def __listclass(self, aClass, ilosc):
        kropki = '.' * ilosc
        if aClass in self.__visited:
            return f'\n{kropki}<klasa {aClass.__name__}, adres {id(aClass)}: (patrz wyżej)'
        else:
            self.__visited[aClass] = True
            genabove = (self.__listclass(c, ilosc + 4) for c in aClass.__bases__)
            return f"\n{kropki}<klasa {aClass.__name__}, adres {id(aClass)}:\n{self.__attrnames(aClass, ilosc)}{''.join(genabove)}{kropki}"
    def __attrnames(self, obj, ilosc):
        spacje = ' ' * (ilosc + 4)
        result = ''
        for attr in sorted(obj.__dict__):
            if attr.startswith('__') and attr.endswith('__'):
                result += spacje + f'{attr} = <>\n'
            else:
                result += spacje + f'{attr}={getattr(obj, attr)}\n'
        return result

In [82]:
class TT(KlasaN, ListTree):
    def __init__(self):
        KlasaN.__init__(self)
        self.data2 = 'jaja'
        self.data3 = 42
    def spam(self):
        pass

if __name__ == '__main__':
    A = TT()
    print(A)

<instancja klasyTT, adres 84450224: 
    _ListTree__visited={}
    atrybutklasyN=dziedziczony
    data2=jaja
    data3=42

....<klasa TT, adres 76107680:
        __doc__ = <>
        __init__ = <>
        __module__ = <>
        spam=<function TT.spam at 0x00000000053860D0>

........<klasa KlasaN, adres 76100128:
            __dict__ = <>
            __doc__ = <>
            __init__ = <>
            __module__ = <>
            __weakref__ = <>
            ham=<function KlasaN.ham at 0x00000000050B9430>

............<klasa object, adres 8790742969168:
                __class__ = <>
                __delattr__ = <>
                __dir__ = <>
                __doc__ = <>
                __eq__ = <>
                __format__ = <>
                __ge__ = <>
                __getattribute__ = <>
                __gt__ = <>
                __hash__ = <>
                __init__ = <>
                __init_subclass__ = <>
                __le__ = <>
                __lt__ = <>
           

### FABRYKI KLAS (I INNYCH OBIEKTÓW):

In [83]:
def factory(klasa, *args):   # utworzenie prostej fabryki obiektow
    return klasa(*args)

In [84]:
class Spam:
    def doit(self, message):
        print(message)

In [85]:
class Person:
    def __init__(self, name, job):
        self.name = name
        self.job = job

In [86]:
object1 = factory(Spam)  #  uzycie funkcji-fabryki w celu stworzenia instancji klasy

In [87]:
object2 = factory(Person, "Zimowit", "szaman")

In [88]:
# druga opcja:
def factory2(objekt, *args, **kwargs):   # dodatkowo przekazuje argumenty ze slowami kluczowymi, typu a=1
    return aply(objekt, args, kwargs)  # funkcja apply przekazuje trzeci argument czyli **kwargs